In [1]:
import cv2
import numpy as np
import vehicle
import time

In [2]:
cap = cv2.VideoCapture('test_data.mp4')
video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

frame_area=video_width*video_height
area_minimum=frame_area/400 #minimum area of a blob to be counted

mid_end=int(4*(video_height/5)) #count blob after it cross (x, mid_end) 
down_limit=video_height

#background substraction
removedBackground=cv2.createBackgroundSubtractorMOG2() 

#initialization
font = cv2.FONT_HERSHEY_SIMPLEX
cars = []
count_car = 0
max_pointer_counter = 5
pointerid = 0

#kernel for opening and closing
kernalOpen = np.ones((5,5),np.uint8)
kernalClose = np.ones((11,11),np.uint8)

while cap.isOpened():
    ret,frame = cap.read();
    
    if ret == True:
        #increase all the counted value of pre-detecting cars
        for i in cars:
            i.increment_value()
            
        # frame pre-processing
        forground_mask=removedBackground.apply(frame)
        _,image=cv2.threshold(forground_mask,200,255,cv2.THRESH_BINARY)
        mask=cv2.morphologyEx(image,cv2.MORPH_OPEN,kernalOpen)
        mask=cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernalClose)
        _, countours,hierarchy=cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
        
        #loop over the detecting contours in each frame
        for cnt in countours:
            area=cv2.contourArea(cnt)
            if area>area_minimum:
                #if a countour have countable area get it's center point
                m=cv2.moments(cnt)
                cx=int(m['m10']/m['m00'])
                cy=int(m['m01']/m['m00'])
                x,y,w,h=cv2.boundingRect(cnt)

                new=True
                #if the y-axis of detected contour inside frame
                if cy in range(down_limit):
                    #loop over existing cars
                    for i in cars:
                        #if the car(x,y) have distance between the looping car then the center just forward not new blob
                        if abs(x - i.getX()) <= w and abs(y - i.getY()) <= h: 
                            new = False
                            #update pre existing center to it's new coordinate
                            i.updateCoords(cx, cy)
                            #after updating if it reach the detection line the car gets counted
                            if i.moving(mid_end)==True:
                                count_car+=1 
                            break
                        #if the state is '1' and y coor > detection line then the centoroid count complete
                        if i.getState()=='1':
                            if i.getY()>down_limit:
                                i.setComplete()
                        # if the center is counted the it gets emoved
                        if i.timeOut():
                            index=cars.index(i)
                            cars.pop(index)
                            del i

                    #if nothing detected then register new coor in the array
                    if new==True:
                        pointer=vehicle.Car(pointerid,cx,cy,max_pointer_counter)
                        cars.append(pointer)
                        pointerid+=1

                cv2.circle(frame,(cx,cy),5,(0,0,255),-1)
                img=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
                
        str_counted='Vehicles: '+str(count_car)
        cv2.line(frame, (0,int(mid_end)), (960,int(mid_end)), (255,127,0), 3) 
        cv2.putText(frame, str_counted, (10, 90), font, 4, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow('Mask',mask)
        cv2.imshow('Frame',frame)
        
        if (cv2.waitKey(10) & 0xFF == ord('q')):
            break
    else:
        break
        
cap.release()
cv2.destroyAllWindows()

In [3]:
count_car

20